In [ ]:
# Importing necessary libraries
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense, Flatten

In [ ]:
# Downloading dataset
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### 1. **clipvalue (Value Clipping)**
- **What it does**: Clips each **individual gradient element** to a maximum/minimum value.
- **Operation**: Applied element-wise to every single number in the gradient tensor.
- **Formula**:  
$$
g_{\text{clipped}} = \max\!\bigl(\min(g,\; \text{clip\_value}),\; -\text{clip\_value}\bigr)
$$
  Where `clip_value` is your threshold (e.g., 0.5).
- **Effect**: Prevents any single weight from getting an update larger than `±clip_value`.
- **Use case**: Less common now. Can be too aggressive as it doesn't consider the overall gradient direction.

**Example**: If `clip_value=0.5` and a gradient is `[1.2, -0.3, -0.8]`, it becomes `[0.5, -0.3, -0.5]`.

### 2. **clipnorm (Norm Clipping)**
- **What it does**: Scales the **entire gradient vector** when its norm exceeds a threshold.
- **Operation**: Works on the whole vector to preserve the gradient direction.
- **Formula**:  
$$
g_{\text{clipped}} = \frac{\max\_norm}{\|g\|_2} \cdot g \quad \text{if} \quad \|g\|_2 > \max\_norm
$$
- **Effect**: Controls the total step size while maintaining the original update direction (just shorter).
- **Use case**: **Most common and recommended**. Preserves the gradient's directional information.

**Example**: If gradients are `[3, 4]` (norm = 5) and `max_norm=2`, they become `[1.2, 1.6]` (norm = 2).

### Quick Comparison
| Feature | **clipvalue** | **clipnorm** |
|---------|--------------|--------------|
| **What it limits** | Individual gradient values | Overall gradient magnitude |
| **Preserves direction** | ❌ No (distorts direction) | ✅ Yes (only scales) |
| **Typical threshold** | 0.1 to 1.0 | 0.5 to 5.0 |
| **Common usage** | Less common today | Standard practice |

In [5]:
# Model Building
model = Sequential([
    Input(shape=[28, 28], batch_size=32),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=10, activation='softmax')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (32, 784)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 128)              │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (32, 10)               │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Model Training
model.compile(
    optimizer = keras.optimizers.Adam(clipvalue=1.0), # All the gradients which either greater than 1 or lesser than -1 are clipped to 1 and -1
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7424 - loss: 17.6699 - val_accuracy: 0.7289 - val_loss: 26.9553
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7792 - loss: 20.3557 - val_accuracy: 0.7597 - val_loss: 29.6934
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7738 - loss: 35.6614 - val_accuracy: 0.7103 - val_loss: 75.9841
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7675 - loss: 51.6642 - val_accuracy: 0.6988 - val_loss: 104.1808
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7727 - loss: 63.8006 - val_accuracy: 0.7633 - val_loss: 75.9723
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.7737 - loss: 79.9983 - val_accuracy: 0.8001 - val_loss: 70.1591
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7737 - loss: 100.7916 - val_accuracy: 0.8179 - val_loss: 80.4404
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.77

### 📐 The Exact Formula
When the gradient vector's norm exceeds `max_norm`, we scale it down:

$$
\mathbf{g} \leftarrow \frac{\text{max\_norm}}{\|\mathbf{g}\|_2} \cdot \mathbf{g} \quad \text{if} \quad \|\mathbf{g}\|_2 > \text{max\_norm}
$$

Here, `||g||₂` in the denominator is calculated as:

$$
\|\mathbf{g}\|_2 = \sqrt{g_1^2 + g_2^2 + \dots + g_n^2}
$$

Where `g₁, g₂, ..., gₙ` are the individual gradient values.

### 🔢 Simple Example
Let’s say your gradients are `g = [3, 4]` and `max_norm = 2`.

1. **Calculate denominator** `||g||₂`:
   $$
   \|\mathbf{g}\|_2 = \sqrt{3^2 + 4^2} = \sqrt{9 + 16} = \sqrt{25} = 5
   $$

2. **Check clipping condition:**
   `5 > 2` → Clipping is needed.

3. **Apply scaling:**
   $$
   \text{Scale factor} = \frac{\text{max\_norm}}{\|\mathbf{g}\|_2} = \frac{2}{5} = 0.4
   $$
   $$
   \text{New gradient} = 0.4 \cdot [3, 4] = [1.2, 1.6]
   $$

4. **Verify new norm:**
   $$
   \|\text{new\_g}\|_2 = \sqrt{1.2^2 + 1.6^2} = \sqrt{1.44 + 2.56} = \sqrt{4} = 2
   $$
   ✅ Exactly equals `max_norm`.

In [ ]:
# Gradient Clipping using `clipnorm`
model = Sequential([
    Input(shape=[28, 28], batch_size=32),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=10, activation='softmax')
])

model.compile(
    optimizer = keras.optimizers.Adam(clipnorm=1.0),  # Keeps gradient directions intact while capping total norm to 1.0
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_test, y_test))

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.6930 - loss: 2.8664 - val_accuracy: 0.7219 - val_loss: 0.8919
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7988 - loss: 0.5859 - val_accuracy: 0.8204 - val_loss: 0.5587
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8185 - loss: 0.5398 - val_accuracy: 0.8135 - val_loss: 0.6045
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8284 - loss: 0.5148 - val_accuracy: 0.8015 - val_loss: 0.5831
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8316 - loss: 0.5077 - val_accuracy: 0.8297 - val_loss: 0.5625
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8368 - loss: 0.4963 - val_accuracy: 0.8248 - val_loss: 0.5283
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8375 - loss: 0.4892 - val_accuracy: 0.8204 - val_loss: 0.5397
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.8423 - loss: 0.4780 - 

### Which to Use?
- **Use `clipnorm` (norm clipping) by default** – it's more mathematically sound as it preserves the gradient direction while controlling step size.
- Use `clipvalue` only in specific cases where you know you need strict element-wise bounds (rare). Working with extream outliers

In practice, most frameworks default to norm clipping because it maintains the optimization trajectory's direction while ensuring stability.

---

### ✅ Advantages (Why It's Useful)

| Advantage | Explanation |
| :--- | :--- |
| **1. Prevents Exploding Gradients** | The primary benefit. Stops weights from receiving giant, destabilizing updates that can cause numerical overflow (NaN loss) or make the model diverge. |
| **2. Stabilizes Training** | Allows the use of higher learning rates by clipping occasional large updates, often leading to faster convergence. |
| **3. Essential for RNNs/Transformers** | Crucial for architectures that process sequences (like LSTMs, GRUs, Transformers) where backpropagation through many time steps (long sequences) can cause gradients to multiply and explode. |
| **4. Handles Noisy or Sparse Data (Overfitting)** | Protects against outlier samples or mini-batches that might produce abnormally large gradients. |

### ❌ Disadvantages & Risks

| Disadvantage | Explanation & Mitigation |
| :--- | :--- |
| **1. Masks Underlying Problems** | Large gradients can be a symptom of poor initialization, too high a learning rate, or a bug. Clipping might let you train a broken model. **Fix**: Use it alongside good practices, not as a substitute. |
| **2. Introduces a Hyperparameter** | The clipping threshold (`max_norm` or `clipvalue`) must be tuned. A bad value can hinder learning (too low) or be ineffective (too high). |
| **3. Can Distort the True Gradient Direction** | Scaling the gradient changes its direction. This alters the optimization path, which could theoretically slow convergence. In practice, the stability gain outweighs this. |
| **4. Not a Cure-All** | It doesn't help with **vanishing gradients** (where gradients become too small). |

---

### 🎯 Where and When to Use It (With Examples)

You should strongly consider gradient clipping in these scenarios:

1.  **Training Recurrent Neural Networks (RNNs)**: This is the **most classic use case**. When you backpropagate through long sequences (many time steps), gradients are multiplied by the same weight matrix repeatedly. This can cause them to grow exponentially (explode).
    *   **Example**: Training an LSTM for text generation or machine translation.

2.  **Training Deep Transformers**: Similar to RNNs, the deep encoder-decoder structure and attention mechanisms in models like BERT or GPT can suffer from exploding gradients, especially in early training.
    *   **Example**: Fine-tuning a large language model (LLM).

3.  **Using Very Deep Networks**: Any very deep feedforward network is at risk. Gradient clipping is a standard safety feature.
    *   **Example**: Training a 100-layer ResNet variant.

4.  **When You See Training Instability**: If your training loss suddenly spikes to `NaN` or shows wild oscillations, exploding gradients are a likely culprit. Gradient clipping is the first line of defense.
    *   **Symptom**: Loss goes from 1.5 → `NaN` in one step.

5.  **When Using Aggressive Optimizers/Settings**: Optimizers like Adam are less prone to explosion but it can still happen. Clipping is wise when using a high learning rate or large batch sizes.

---

**Choosing a Threshold (`max_norm`)**:
*   **Common Starting Points**: Values between `0.5` and `10.0` are typical. `1.0` or `5.0` are very common defaults.
*   **How to Tune**: Monitor your gradient norms during a few training steps. Set the threshold slightly above the **typical range** you observe, so it only clips the pathological outliers.